In [2]:
from zipfile import ZipFile
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

archive = ZipFile('Online Retail.zip', 'r')
file = archive.open('Online Retail.xlsx')
df = pd.read_excel(file)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240992 entries, 0 to 240991
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    240992 non-null  object        
 1   StockCode    240992 non-null  object        
 2   Description  240091 non-null  object        
 3   Quantity     240992 non-null  int64         
 4   InvoiceDate  240992 non-null  datetime64[ns]
 5   UnitPrice    240992 non-null  float64       
 6   CustomerID   173555 non-null  float64       
 7   Country      240992 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 14.7+ MB


In [4]:
df.groupby('Country')['InvoiceNo'].count().sort_values(ascending=False)

Country
United Kingdom          221186
Germany                   4209
France                    3642
EIRE                      3080
Spain                     1147
Netherlands               1142
Belgium                    933
Switzerland                735
Australia                  642
Portugal                   624
Norway                     378
Channel Islands            368
Cyprus                     353
Finland                    312
Italy                      309
Japan                      251
Hong Kong                  200
Sweden                     200
Poland                     187
Denmark                    184
Austria                    127
Singapore                  118
Iceland                    102
Greece                      85
Unspecified                 83
Canada                      68
Malta                       47
Lebanon                     45
Israel                      38
Lithuania                   35
European Community          32
Brazil                      32


In [42]:
#priprava tabulky pro Svedsko

basket_sw = df[df['Country'] == 'Sweden'].loc[:, ['Description', 'InvoiceNo', 'Quantity']]
basket_sw = basket_sw.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack(1, fill_value=0)

def number_to_one(entry):
    if entry >= 1:
        return 1
    else:
        return 0
       

basket_sw = basket_sw.applymap(number_to_one)
basket_sw.head()

Description,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,3 PIECE SPACEBOY COOKIE CUTTER SET,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,5 HOOK HANGER MAGIC TOADSTOOL,60 CAKE CASES DOLLY GIRL DESIGN,...,TREASURE ISLAND BOOK BOX,VICTORIAN SEWING KIT,VINTAGE HEADS AND TAILS CARD GAME,WALL TIDY RETROSPOT,WOODEN BOX OF DOMINOES,WOODEN CROQUET GARDEN SET,WOODEN HAPPY BIRTHDAY GARLAND,WOODEN OWLS LIGHT GARLAND,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP ALPHABET DESIGN
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
538848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539338,0,0,0,0,0,1,0,0,1,1,...,0,1,0,0,0,0,0,0,1,0
540040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
542428,0,0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
542911,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [53]:
#doporuceni kombinaci produktu pro Svedsko, vysledky jsou serazene, vrchni radky jsou dvojice produktu, ktere by mohly byt nabizene spolu.

frequent_items_sw = apriori(basket_sw, min_support = 0.05, use_colnames = True)
rules_sw = association_rules(frequent_items_sw, metric = 'lift', min_threshold=1)
rules_sw.sort_values(['lift', 'confidence'], ascending=[False, False]).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(36 DOILIES DOLLY GIRL),(60 CAKE CASES DOLLY GIRL DESIGN),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
1,(60 CAKE CASES DOLLY GIRL DESIGN),(36 DOILIES DOLLY GIRL),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
2,(36 DOILIES DOLLY GIRL),(ASSORTED BOTTLE TOP MAGNETS ),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
3,(ASSORTED BOTTLE TOP MAGNETS ),(36 DOILIES DOLLY GIRL),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
8,(36 DOILIES DOLLY GIRL),(PACK OF 60 PINK PAISLEY CAKE CASES),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
9,(PACK OF 60 PINK PAISLEY CAKE CASES),(36 DOILIES DOLLY GIRL),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
10,(36 DOILIES DOLLY GIRL),(PACK OF 60 SPACEBOY CAKE CASES),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
11,(PACK OF 60 SPACEBOY CAKE CASES),(36 DOILIES DOLLY GIRL),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
12,(36 DOILIES DOLLY GIRL),(PACK OF 72 RETROSPOT CAKE CASES),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf
13,(PACK OF 72 RETROSPOT CAKE CASES),(36 DOILIES DOLLY GIRL),0.095238,0.095238,0.095238,1.0,10.5,0.086168,inf


In [50]:
#priprava tabulky pro Spanelsko

basket_sp = df[df['Country'] == 'Spain'].loc[:, ['Description', 'InvoiceNo', 'Quantity']]
basket_sp = basket_sp.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack(1, fill_value=0)

def number_to_one(entry):
    if entry >= 1:
        return 1
    else:
        return 0
       

basket_sp = basket_sp.applymap(number_to_one)
basket_sp.head()

Description,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,...,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP MONSTER FUN,WRAP RED APPLES,"WRAP, BILLBOARD FONTS DESIGN",YELLOW COAT RACK PARIS FASHION,YELLOW FLOWERS FELT HANDBAG KIT,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536944,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
538525,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
540469,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [54]:
#doporuceni kombinaci produktu pro Spanelsko, vysledky jsou serazene, vrchni radky jsou dvojice produktu, ktere by mohly byt nabizene spolu.

frequent_items_sp = apriori(basket_sp, min_support = 0.05, use_colnames = True)
rules_sp = association_rules(frequent_items_sp, metric = 'lift', min_threshold=1)
rules_sp.sort_values(['lift', 'confidence'], ascending=[False, False]).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(36 FOIL HEART CAKE CASES),(RETROSPOT TEA SET CERAMIC 11 PC ),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
3,(RETROSPOT TEA SET CERAMIC 11 PC ),(36 FOIL HEART CAKE CASES),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
92,(BLUE HARMONICA IN BOX ),(RED HARMONICA IN BOX ),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
93,(RED HARMONICA IN BOX ),(BLUE HARMONICA IN BOX ),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
94,(WOODEN BOX OF DOMINOES),(BLUE HARMONICA IN BOX ),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
95,(BLUE HARMONICA IN BOX ),(WOODEN BOX OF DOMINOES),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
188,(SILK PURSE BABUSHKA PINK),(DOORMAT FANCY FONT HOME SWEET HOME),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
189,(DOORMAT FANCY FONT HOME SWEET HOME),(SILK PURSE BABUSHKA PINK),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
196,(PINK HEART SHAPE EGG FRYING PAN),(ENAMEL PINK COFFEE CONTAINER),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
197,(ENAMEL PINK COFFEE CONTAINER),(PINK HEART SHAPE EGG FRYING PAN),0.066667,0.066667,0.066667,1.0,15.0,0.062222,inf
